In [2]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer as SIA
import pandas as pd
import praw

In [3]:
sia = SIA()
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Adria\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Get Headlines From Reddit

In [9]:
user_agent = "Scraper"
reddit = praw.Reddit(
    client_id = "aKfBu6Agrx3kW_ElCYh5nA",
    client_secret = "Ngao_Oo_eW8DFPy0qzjFhi5qSoBDHg",
    user_agent = user_agent
)

headlines = set()
for submission in reddit.subreddit('politics').hot(limit=None):
    headlines.add(submission.title)

for submission in reddit.subreddit('news').hot(limit=None):
    headlines.add(submission.title)

for submission in reddit.subreddit('worldnews').hot(limit=None):
    headlines.add(submission.title)
    
print(len(headlines))

1687


In [10]:
df = pd.DataFrame(headlines)
df.head()

,0
0,Climate change: Russia burns off gas as Europe...
1,The Origin of Student Debt: Reagan Adviser War...
2,Chip shops face 'extinction' amid cost of livi...
3,The problem(s) with the GOP’s new ‘election in...
4,Tesla loses challenge to California agency sui...


In [5]:
df.to_csv('headlines.csv', header = False, encoding = 'utf-8', index = False)

Perform Sentiment Analysis

In [11]:
results = []

for line in headlines:
    polarity_score = sia.polarity_scores(line)   #Outputs dict
    polarity_score['headline'] = line
    results.append(polarity_score)

print(results[0:3])

[{'neg': 0.0, 'neu': 0.826, 'pos': 0.174, 'compound': 0.2732, 'headline': 'Climate change: Russia burns off gas as Europe&#x27;s energy bills rocket'}, {'neg': 0.333, 'neu': 0.433, 'pos': 0.234, 'compound': -0.3182, 'headline': 'The Origin of Student Debt: Reagan Adviser Warned Free College Would Create a Dangerous “Educated Proletariat”'}, {'neg': 0.339, 'neu': 0.661, 'pos': 0.0, 'compound': -0.6249, 'headline': "Chip shops face 'extinction' amid cost of living crisis"}]


In [12]:
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,headline
0,0.000,0.826,0.174,0.2732,Climate change: Russia burns off gas as Europe...
1,0.333,0.433,0.234,-0.3182,The Origin of Student Debt: Reagan Adviser War...
2,0.339,0.661,0.000,-0.6249,Chip shops face 'extinction' amid cost of livi...
3,0.000,1.000,0.000,0.0000,The problem(s) with the GOP’s new ‘election in...
4,0.443,0.458,0.099,-0.5423,Tesla loses challenge to California agency sui...


Label and Create new CSV File

In [24]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < 0.2, 'label'] = -1
df.loc[df['compound'] == 0, 'label'] = 0
df.head()

,neg,neu,pos,compound,headline,label
0,0.000,0.826,0.174,0.2732,Climate change: Russia burns off gas as Europe...,1
1,0.333,0.433,0.234,-0.3182,The Origin of Student Debt: Reagan Adviser War...,-1
2,0.339,0.661,0.000,-0.6249,Chip shops face 'extinction' amid cost of livi...,-1
3,0.000,1.000,0.000,0.0000,The problem(s) with the GOP’s new ‘election in...,0
4,0.443,0.458,0.099,-0.5423,Tesla loses challenge to California agency sui...,-1


In [27]:
df2 = df[['label', 'headline']]
df2.head()
df2.to_csv('Labeled_Headlines.csv', header = False, encoding = 'utf-8', index = False)